# Convert Kinetics Library to Training Reactions Script

Specify the kinetics library name below and run the script.  It automatically overwrites the training reactions files it needs to.  Then you should commit those files.

This script only trains safely.  In other words, if a single match from an RMG family is found, a training reaction is created.  Sometimes, there are no matches from RMG reaction families, or multiple matches.  This indicates an error that requires manual fixing, and a printout is given in the script.

In [1]:
# Set libraries to load reactions from; set to None to load all libraries
# libraries = ['PFAS_HPL']
libraries = ['PFAS_HPL_Test']
# Set families to add training reactions to; either 'default' or a list, e.g. ['R_Addition_MultipleBond']
families = ['CF_Radical_Formation_1', 'OF_Radical_Formation', 'Lactone_to_Perfluoroalkene', 'CO2_Elimination_From_Lactone', 'CO2_Elimination_From_Carboxylic_Acid', 'CO2_Elimination_From_Lactone_Ether', 'CO_Elimination_From_Lactone', 'CO_CF_bond_dissociation', 'PFAS_Hydrolysis', 'Lactone_Formation', 'CO2_Elimination_From_PFCA_Radical']

# Specify whether to plot kinetics comparisons
compare_kinetics = False

# Specify whether to print library reactions which don't fit in the specified families
# This can result in a lot of unnecessary output if only using a few families
show_all = False

# Specify whether to prioritize aromatic resonance structures to reduce cases of multiple matches
filter_aromatic = True

# Specify whether to use verbose comments when averaging tree
verbose_comments = True

In [2]:
from rmgpy import settings
from rmgpy.data.rmg import RMGDatabase
from kinetics_library_to_training_tools_modified import *
import kinetics_library_to_training_tools_modified# import process_reactions
from base64 import b64encode
from IPython.display import display, HTML


In [ ]:
from importlib import reload

In [3]:
thermolibs = [
'C1_C2_Fluorine', #putting Siddha's as most trusted because Caroline used this thermo for calcs
'PFCA_thermo',
'NCSU_C2_C8_PFAS', #using this as second trusted thermo because has good estimates for higher C pfas
'Fluorine',
'primaryThermoLibrary',
'FFCM1(-)',
'halogens',
'CHOF_G4',
'CHOCl_G4',
'CHOBr_G4',
'CHOFCl_G4',
'CHOFBr_G4',
'CHOFClBr_G4',
'DFT_QCI_thermo',
'2-BTP_G4',
'thermo_DFT_CCSDTF12_BAC',
'SulfurHaynes'
]


## Step 1: Load RMG-database with specified libraries and families

In [4]:
database = RMGDatabase()
database.load(
    path = settings['database.directory'],
    thermo_libraries = thermolibs,  # Can add others if necessary
    kinetics_families = families,
    reaction_libraries = libraries,
    kinetics_depositories = ['training'],
)
# If we want accurate kinetics comparison, add existing training reactions and fill tree by averaging
if compare_kinetics:
    for family in database.kinetics.families.values():
        if not family.auto_generated:
            family.add_rules_from_training(thermo_database=database.thermo)
            family.fill_rules_by_averaging_up(verbose=verbose_comments)

In [5]:
database.kinetics.families.keys()

dict_keys(['CF_Radical_Formation_1', 'CO2_Elimination_From_Carboxylic_Acid', 'CO2_Elimination_From_Lactone', 'CO2_Elimination_From_Lactone_Ether', 'CO2_Elimination_From_PFCA_Radical', 'CO_CF_bond_dissociation', 'CO_Elimination_From_Lactone', 'Lactone_Formation', 'Lactone_to_Perfluoroalkene', 'OF_Radical_Formation', 'PFAS_Hydrolysis'])

## Step 2a: Generate library reactions from families to get proper labels

In [ ]:
reload(kinetics_library_to_training_tools_modified)

In [6]:
master_dict, multiple_dict, lib_fam_rxn_dict = kinetics_library_to_training_tools_modified.process_reactions(database,
                                               libraries,
                                               list(database.kinetics.families.keys()),
                                               compare_kinetics=compare_kinetics,
                                               show_all=show_all,
                                               filter_aromatic=filter_aromatic)

[Species(label="CF2H", molecule=[Molecule(smiles="F[CH]F")], molecular_weight=(51.0154,'amu'))] [Species(label="CF", molecule=[Molecule(smiles="[C]F")], molecular_weight=(31.009,'amu')), Species(label="HF", molecule=[Molecule(smiles="F")], molecular_weight=(20.0064,'amu'))]


In [7]:
master_dict['PFAS_HPL'].keys()

KeyError: 'PFAS_HPL'

In [ ]:
reaction_dict = master_dict['PFAS_HPL']

In [ ]:
#these were the ones that were matched: 
matched_families = list(master_dict['PFAS_HPL'].keys())

for fmly in matched_families: 
    family_name = fmly
    analyze_per_family(family_name, lib_fam_rxn_dict, reaction_dict, database, compare_kinetics=False)
    

In [ ]:
#these are the reactions that were unmatched
unmatched, matched = pick_out_unmatched_rxns(database, libraries, list(database.kinetics.families.keys()))

In [ ]:
print(len(matched), len(unmatched))

In [ ]:
family_name = 'OF_Radical_Formation'
analyze_per_family(family_name, lib_fam_rxn_dict, reaction_dict, database, compare_kinetics=False)

In [ ]:
family_name = 'Lactone_Formation'
analyze_per_family(family_name, lib_fam_rxn_dict, reaction_dict, database, compare_kinetics=False)

In [ ]:
family_name = '1,2_Insertion_carbene'
analyze_per_family(family_name, lib_fam_rxn_dict, reaction_dict, database, compare_kinetics=True)


In [ ]:
family_name = 'R_Addition_MultipleBond'
analyze_per_family(family_name, lib_fam_rxn_dict, reaction_dict, database, compare_kinetics=True)


In [ ]:
family_name = 'XY_Addition_MultipleBond'
analyze_per_family(family_name, lib_fam_rxn_dict, reaction_dict, database, compare_kinetics=True)


In [ ]:
family_name = 'Singlet_Carbene_Intra_Disproportionation'
analyze_per_family(family_name, lib_fam_rxn_dict, reaction_dict, database, compare_kinetics=True)


In [ ]:
family_name = '1,2_Insertion_CO'
analyze_per_family(family_name, lib_fam_rxn_dict, reaction_dict, database, compare_kinetics=True)


In [ ]:
family_name = '1+2_Cycloaddition'
analyze_per_family(family_name, lib_fam_rxn_dict, reaction_dict, database, compare_kinetics=True)


In [ ]:
family_name = '1,3_Insertion_CO2'
analyze_per_family(family_name, lib_fam_rxn_dict, reaction_dict, database, compare_kinetics=True)


In [ ]:
family_name = '1,3_sigmatropic_rearrangement'
analyze_per_family(family_name, lib_fam_rxn_dict, reaction_dict, database, compare_kinetics=True)


In [ ]:
#find the reactions without matches and save them to a file


## Step 2b (optional): Review and select reactions to be added

In [ ]:
review_reactions(master_dict, prompt=True)

## Step 2c (optional): Manual processing for reactions with multiple matches

In [ ]:
manual_selection(master_dict, multiple_dict, database)

## Step 2d: Final review of reactions to be added

In [ ]:
review_reactions(master_dict, prompt=False)

## Step 3: Write the new training reactions to the database

In [ ]:
for library_name, reaction_dict in master_dict.items():
    library = database.kinetics.libraries[library_name]
    
    for family_name, reaction_list in reaction_dict.items():
        print('Adding training reactions from {0} to {1}...'.format(library_name, family_name))

        family = database.kinetics.families[family_name]
        try:
            depository = family.get_training_depository()
        except:
            raise Exception('Unable to find training depository in {0}. Check that one exists.'.format(family_name))

        print('Training depository previously had {} rxns. Now adding {} new rxn(s).'.format(len(depository.entries), len(reaction_list)))

        ref_list = []
        type_list = []
        short_list = []
        long_list = []
        
        for reaction in reaction_list:
            # Get the original entry to retrieve metadata
            orig_entry = library.entries[reaction.index]
            short_desc = orig_entry.short_desc
            long_desc = 'Training reaction from kinetics library: {0}\nOriginal entry: {1}'.format(library_name, orig_entry.label)
            if orig_entry.long_desc:
                long_desc += '\n' + orig_entry.long_desc
            
            ref_list.append(orig_entry.reference)
            type_list.append(orig_entry.reference_type)
            short_list.append(short_desc)
            long_list.append(long_desc)
            
        family.save_training_reactions(
            reaction_list,
            reference=ref_list,
            reference_type=type_list,
            short_desc=short_list,
            long_desc=long_list,
        )